In [25]:
'''
这个code的目的是用neurosketch 的数据来检测现在在realtime data里面发现的issue：也就是ceiling有时候竟然比floor更小
这个code的运行逻辑是
用neurosketch前五个run训练2 way classifiers，然后用最后一个run来计算ceiling和floor的值，看是否合理
'''


'''
purpose:
    find the best performed mask from the result of aggregate_greedy.py and save as chosenMask
    train all possible pairs of 2way classifiers and save for evidence calculation
    load saved classifiers and calculate different forms of evidence
steps:
    load the result of aggregate_greedy.py
    display the result of aggregate_greedy.py
    find the best performed ROI for each subject and display the accuracy of each subject, save the best performed ROI as chosenMask
    load the functional and behavior data and choseMask and train all possible pairs of 2way classifiers
    calculate the evidence floor and ceil for each subject and display different forms of evidences.
    

'''




'''
load the result of aggregate_greedy.py
'''
# To visualize the greedy result starting for 31 ROIs, in total 25 subjects.
import os
os.chdir("/gpfs/milgram/project/turk-browne/projects/rtTest/kp_scratch/")
from glob import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle5 as pickle
import subprocess
import numpy as np
import os
print(f"conda env={os.environ['CONDA_DEFAULT_ENV']}") 
import numpy as np
import nibabel as nib
import sys
import time
import pandas as pd
from sklearn.linear_model import LogisticRegression
import itertools
import pickle
import subprocess
from subprocess import call
import warnings
warnings.filterwarnings("ignore")

workingDir="/gpfs/milgram/project/turk-browne/projects/rtTest/"

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
roiloc="schaefer2018"
dataSource="neurosketch"
subjects_correctly_aligned=['1206161','0119173','1206162','1130161','1206163','0120171','0111171','1202161','0125172','0110172','0123173','0120173','0110171','0119172','0124171','0123171','1203161','0118172','0118171','0112171','1207162','0117171','0119174','0112173','0112172']
subjects=subjects_correctly_aligned
N=25
workingPath="/gpfs/milgram/project/turk-browne/projects/rtTest/"
GreedyBestAcc=np.zeros((len(subjects),N+1))
GreedyBestAcc[GreedyBestAcc==0]=None
GreedyBestAcc={}
numberOfROIs={}
for ii,subject in enumerate(subjects):
    # try:
    #     GreedyBestAcc[ii,N]=np.load(workingPath+"./{}/{}/output/uniMaskRanktag2_top{}.npy".format(roiloc, subject, N))
    # except:
    #     pass
    t=np.load(workingPath+"./{}/{}/output/uniMaskRanktag2_top{}.npy".format(roiloc, subject, N))
    GreedyBestAcc[subject]=[np.float(t)]
    numberOfROIs[subject]=[N]
    # for len_topN_1 in range(N-1,0,-1):
    for len_topN in range(1,N):
        # Wait(f"./tmp/{subject}_{N}_{roiloc}_{dataSource}_{len_topN_1}.pkl")
        try:
            # {当前的被试}_{greedy开始的ROI数目，也就是25}_{mask的种类schaefer2018}_{数据来源neurosketch}_{当前的 megaROI 包含有的数目}
            di = load_obj(f"./tmp__folder/{subject}_{N}_{roiloc}_{dataSource}_{len_topN}")
            GreedyBestAcc[subject].append(np.float(di['bestAcc']))
            numberOfROIs[subject].append(len_topN)
            # GreedyBestAcc[ii,len_topN] = di['bestAcc']
            
        except:
            pass


# '''
# to load the imtermediate results from greedy code to examine the system
# '''
# def wait(tmpFile):
#     while not os.path.exists(tmpFile+'_result.npy'):
#         time.sleep(5)
#         print(f"waiting for {tmpFile}_result.npy\n")
#     return np.load(tmpFile+'_result.npy')

# subject= '0119173' #sys.argv[1]
# sub_id = [i for i,x in enumerate(subjects) if x == subject][0]
# intermediate_result=np.zeros((N+1,N+1))
# # 应该有多少？25个24ROI，2个1ROI，24个
# for i in range(N,1,-1):
#     for j in range(i):
#         tmpFile=f"./tmp__folder/{subject}_{N}_{roiloc}_{dataSource}_{i}_{j}"
#         sl_result=wait(tmpFile)
#         intermediate_result[i,j]=sl_result

# # _=plt.imshow(intermediate_result)
# #最后一行是25个24ROI，第2行是2个1ROI

'''
display the result of aggregate_greedy.py
'''
# GreedyBestAcc=GreedyBestAcc.T
# plt.imshow(GreedyBestAcc)
# _=plt.figure()
# for i in range(GreedyBestAcc.shape[0]):
#     plt.scatter([i]*GreedyBestAcc.shape[1],GreedyBestAcc[i,:],c='g',s=2)
# plt.plot(np.arange(GreedyBestAcc.shape[0]),np.nanmean(GreedyBestAcc,axis=1))
# # plt.ylim([0.19,0.36])
# # plt.xlabel("number of ROIs")
# # plt.ylabel("accuracy")
# _=plt.figure()
# for j in range(GreedyBestAcc.shape[1]):
#     plt.plot(GreedyBestAcc[:,j])


# GreedyBestAcc=GreedyBestAcc.T
# _=plt.figure()
# plt.imshow(GreedyBestAcc)

'''
find the best performed ROI for each subject and display the accuracy of each subject, save the best performed ROI as chosenMask
'''
#find best ID for each subject
bestID={}
for ii,subject in enumerate(subjects):
    t=GreedyBestAcc[subject]
    bestID[subject] = numberOfROIs[subject][np.where(t==np.nanmax(t))[0][0]] #bestID 指的是每一个subject对应的最好的megaROI包含的ROI的数目
chosenMask={}
for subject in bestID:
    # best ID  
    # {当前的被试}_{greedy开始的ROI数目，也就是25}_{mask的种类schaefer2018}_{数据来源neurosketch}_{最好的megaROI 包含有的数目}
    di = load_obj(f"./tmp__folder/{subject}_{N}_{roiloc}_{dataSource}_{bestID[subject]}")
    chosenMask[subject] = di['bestROIs']

def getMask(topN, subject):
    workingDir="/gpfs/milgram/project/turk-browne/projects/rtTest/"
    for pn, parc in enumerate(topN):
        _mask = nib.load(workingDir+"/{}/{}/{}".format(roiloc, subject, parc))
        aff = _mask.affine
        _mask = _mask.get_data()
        _mask = _mask.astype(int)
        # say some things about the mask.
        mask = _mask if pn == 0 else mask + _mask
        mask[mask>0] = 1
    return mask

for sub in chosenMask:
    mask=getMask(chosenMask[sub], sub)
    # if not os.path.exists(f"{workingDir}/{roiloc}/{sub}/chosenMask.npy"):
    np.save(f"{workingDir}/{roiloc}/{sub}/chosenMask",mask)
    

from scipy.stats import zscore
def normalize(X):
    _X=X.copy()
    _X = zscore(_X, axis=0)
    _X[np.isnan(_X)]=0
    return _X

def mkdir(folder):
    if not os.path.isdir(folder):
        os.mkdir(folder)

'''
calculate the evidence floor and ceil for each subject and display different forms of evidences.
'''
def morphingTarget(subject,testRun=6):
    '''
    purpose:
        get the morphing target function
    steps:
        load train clf
        load brain data and behavior data
        get the morphing target function
            evidence_floor is C evidence for CD classifier(can also be D evidence for CD classifier)
            evidence_ceil  is A evidence in AC and AD classifier
    '''

    import os
    import numpy as np
    import pandas as pd
    import joblib
    import nibabel as nib


    phasedict = dict(zip([1,2,3,4,5,6],["12", "12", "34", "34", "56", "56"]))
    imcodeDict={"A": "bed", "B": "Chair", "C": "table", "D": "bench"}
    if 'milgram' in os.getcwd():
        main_dir='/gpfs/milgram/project/turk-browne/projects/rtTest/'
    else:
        main_dir='/Users/kailong/Desktop/rtTest'

    working_dir=main_dir
    os.chdir(working_dir)

    funcdata = "/gpfs/milgram/project/turk-browne/jukebox/ntb/projects/sketchloop02/subjects/{sub}_neurosketch/data/nifti/realtime_preprocessed/{sub}_neurosketch_recognition_run_{run}.nii.gz"
    metadata = "/gpfs/milgram/project/turk-browne/jukebox/ntb/projects/sketchloop02/data/features/recog/metadata_{sub}_V1_{phase}.csv"

    metas = []

    run=testRun
    print(run, end='--')
    # retrieve from the dictionary which phase it is, assign the session
    phase = phasedict[run]
    ses = 1
    
    # Build the path for the preprocessed functional data
    this4d = funcdata.format(ses=ses, run=run, phase=phase, sub=subject)
    
    # Read in the metadata, and reduce it to only the TR values from this run, add to a list
    thismeta = pd.read_csv(metadata.format(ses=ses, run=run, phase=phase, sub=subject))
    if dataSource == "neurosketch":
        _run = 1 if run % 2 == 0 else 2
    else:
        _run = run
    thismeta = thismeta[thismeta['run_num'] == int(_run)]
    
    if dataSource == "realtime":
        TR_num = list(thismeta.TR.astype(int))
        labels = list(thismeta.Item)
        labels = [imcodeDict[label] for label in labels]
    else:
        TR_num = list(thismeta.TR_num.astype(int))
        labels = list(thismeta.label)
    
    print("LENGTH OF TR: {}".format(len(TR_num)))
    # Load the functional data
    runIm = nib.load(this4d)
    affine_mat = runIm.affine
    runImDat = runIm.get_fdata()
    
    # Use the TR numbers to select the correct features
    features = [runImDat[:,:,:,n+3] for n in TR_num]
    features = np.array(features)
    chosenMask = np.load(f"/gpfs/milgram/project/turk-browne/projects/rtTest/schaefer2018/{subject}/chosenMask.npy")
    features = features[:, chosenMask==1]
    print("shape of features", features.shape, "shape of mask", mask.shape)
    # featmean = features.mean(1).mean(1).mean(1)[..., None,None,None] #features.mean(1)[..., None]
    # features = features - featmean
    # features = features - features.mean(0)
    features = normalize(features)
    # features = np.expand_dims(features, 0)
    
    # Append both so we can use it later
    # metas.append(labels)
    # metas['label']

    t=pd.DataFrame()
    t['label']=labels
    t["run_num"]=run
    behav_data=t
    
    runs = features

    
    dimsize = runIm.header.get_zooms()
    
    brain_data = runs
    print(brain_data.shape)
    print(behav_data.shape)
    FEAT=brain_data
    print(f"FEAT.shape={FEAT.shape}")
    META=behav_data

    # convert item colume to label colume
    imcodeDict={
    'A': 'bed',
    'B': 'chair',
    'C': 'table',
    'D': 'bench'}


    def classifierEvidence(clf,X,Y):
        ID=np.where((clf.classes_==Y)*1==1)[0][0]
        Evidence=(X@clf.coef_.T+clf.intercept_) if ID==1 else (-(X@clf.coef_.T+clf.intercept_))
        # Evidence=(X@clf.coef_.T+clf.intercept_) if ID==0 else (-(X@clf.coef_.T+clf.intercept_))
        return np.asarray(Evidence)

    A_ID = (META['label']=='bed')
    X = FEAT[A_ID]


    model_folder = f"{working_dir}{roiloc}/{subject}/clf/"

    store="\n"
    print("floor")
    # D evidence for AD_clf when A is presented.
    Y = 'bench'
    AD_clf=joblib.load(model_folder +'bedchair_bedbench.joblib') # These 4 clf are the same:   bedchair_bedbench.joblib bedtable_bedbench.joblib benchchair_benchbed.joblib benchtable_benchbed.joblib
    AD_D_evidence = classifierEvidence(AD_clf,X,Y)
    evidence_floor = np.mean(AD_D_evidence)
    print(f"D evidence for AD_clf when A is presented={evidence_floor}")
    store=store+f"D evidence for AD_clf when A is presented={evidence_floor}"

    # C evidence for AC_clf when A is presented.
    Y = 'table'
    AC_clf=joblib.load(model_folder +'benchtable_tablebed.joblib') # These 4 clf are the same:   bedbench_bedtable.joblib bedchair_bedtable.joblib benchtable_tablebed.joblib chairtable_tablebed.joblib
    AC_C_evidence = classifierEvidence(AC_clf,X,Y)
    evidence_floor = np.mean(AC_C_evidence)
    print(f"C evidence for AC_clf when A is presented={evidence_floor}")
    store=store+"\n"+f"C evidence for AC_clf when A is presented={evidence_floor}"

    # D evidence for CD_clf when A is presented.
    Y = 'bench'
    CD_clf=joblib.load(model_folder +'bedbench_benchtable.joblib') # These 4 clf are the same: bedbench_benchtable.joblib bedtable_tablebench.joblib benchchair_benchtable.joblib chairtable_tablebench.joblib
    CD_D_evidence = classifierEvidence(CD_clf,X,Y)
    evidence_floor = np.mean(CD_D_evidence)
    print(f"D evidence for CD_clf when A is presented={evidence_floor}")
    store=store+"\n"+f"D evidence for CD_clf when A is presented={evidence_floor}"

    # C evidence for CD_clf when A is presented.
    Y = 'table'
    CD_clf=joblib.load(model_folder +'bedbench_benchtable.joblib') # These 4 clf are the same: bedbench_benchtable.joblib bedtable_tablebench.joblib benchchair_benchtable.joblib chairtable_tablebench.joblib
    CD_C_evidence = classifierEvidence(CD_clf,X,Y)
    evidence_floor = np.mean(CD_C_evidence)
    print(f"C evidence for CD_clf when A is presented={evidence_floor}")
    store=store+"\n"+f"C evidence for CD_clf when A is presented={evidence_floor}"


    print("ceil")
    store=store+"\n"+"ceil"
    # evidence_ceil  is A evidence in AC and AD classifier
    Y = 'bed'
    AC_clf=joblib.load(model_folder +'benchtable_tablebed.joblib') # These 4 clf are the same:   bedbench_bedtable.joblib bedchair_bedtable.joblib benchtable_tablebed.joblib chairtable_tablebed.joblib
    AC_A_evidence = classifierEvidence(AC_clf,X,Y)
    evidence_ceil1 = AC_A_evidence
    print(f"A evidence in AC_clf when A is presented={np.mean(evidence_ceil1)}")
    store=store+"\n"+f"A evidence in AC_clf when A is presented={np.mean(evidence_ceil1)}"

    Y = 'bed'
    AD_clf=joblib.load(model_folder +'bedchair_bedbench.joblib') # These 4 clf are the same:   bedchair_bedbench.joblib bedtable_bedbench.joblib benchchair_benchbed.joblib benchtable_benchbed.joblib
    AD_A_evidence = classifierEvidence(AD_clf,X,Y)
    evidence_ceil2 = AD_A_evidence
    print(f"A evidence in AD_clf when A is presented={np.mean(evidence_ceil2)}")
    store=store+"\n"+f"A evidence in AD_clf when A is presented={np.mean(evidence_ceil2)}"

    # evidence_ceil = np.mean(evidence_ceil1)
    # evidence_ceil = np.mean(evidence_ceil2)
    evidence_ceil = np.mean((evidence_ceil1+evidence_ceil2)/2)
    print(f"evidence_ceil={evidence_ceil}")
    store=store+"\n"+f"evidence_ceil={evidence_ceil}"
    ceil,floor=evidence_ceil,evidence_floor
    mu = (ceil+floor)/2
    sig = (ceil-floor)/2.3548
    print(f"floor={floor}, ceil={ceil}")
    print(f"mu={mu}, sig={sig}")

    store=store+"\n"+f"floor={floor}, ceil={ceil}"
    store=store+"\n"+f"mu={mu}, sig={sig}"




    BC_clf=joblib.load(model_folder +'benchchair_chairtable.joblib') # These 4 clf are the same: bedbench_benchtable.joblib bedtable_tablebench.joblib benchchair_benchtable.joblib chairtable_tablebench.joblib
    BD_clf=joblib.load(model_folder +'bedchair_chairbench.joblib') # These 4 clf are the same: bedbench_benchtable.joblib bedtable_tablebench.joblib benchchair_benchtable.joblib chairtable_tablebench.joblib
    Y = 'chair'
    # imcodeDict={
    # 'A': 'bed',
    # 'B': 'chair',
    # 'C': 'table',
    # 'D': 'bench'}
    print(f"classifierEvidence(BC_clf,FEAT,Y)={classifierEvidence(BC_clf,FEAT,Y)}")
    print(f"classifierEvidence(BD_clf,FEAT,Y)={classifierEvidence(BD_clf,FEAT,Y)}")





    # # convert item colume to label colume
    # imcodeDict={
    # 'A': 'bed',
    # 'B': 'chair',
    # 'C': 'table',
    # 'D': 'bench'}
    def testMorphParamFor(A): #A='bed''chair'...
        A_ID = (META['label']==A)
        X = FEAT[A_ID]


        # 这里的X是选择的testRun里面的所有的展示A的trials
        BC_B_evidence = classifierEvidence(BC_clf,X,Y)
        BD_B_evidence = classifierEvidence(BD_clf,X,Y)
        print(f"BC_B_evidence={BC_B_evidence}")
        print(f"BD_B_evidence={BD_B_evidence}")
        B_evidence = (BC_B_evidence+BD_B_evidence)/2
        print(f"B_evidence={B_evidence}")
        print(f"mu={mu}, sig={sig}")
        



        def gaussian(x, mu, sig):
            # mu and sig is determined before each neurofeedback session using 2 recognition runs.
            return np.round(1+18*(1 - np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))))) # map from (0,1) -> [1,19]
        morphParam=np.mean(gaussian(B_evidence, mu, sig))
        # B_evidences.append(B_evidence)
        print(f"morphParam={morphParam}")
        return f"morphParam for {A} = {morphParam}"

    store=store+"\n"+testMorphParamFor('bed')
    store=store+"\n"+testMorphParamFor('chair')
    store=store+"\n"+testMorphParamFor('table')
    store=store+"\n"+testMorphParamFor('bench')

    return evidence_floor, evidence_ceil,store



'''
load the functional and behavior data and choseMask and train all possible pairs of 2way classifiers
''' 
# def minimalClass(subject):
subject=subjects[2]
'''
purpose: 
    train offline models

steps:
    load preprocessed and aligned behavior and brain data 
    select data with the wanted pattern like AB AC AD BC BD CD 
    train correspondng classifier and save the classifier performance and the classifiers themselves.

'''

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import joblib
import nibabel as nib
import itertools
from sklearn.linear_model import LogisticRegression

def gaussian(x, mu, sig):
    # mu and sig is determined before each neurofeedback session using 2 recognition runs.
    return round(1+18*(1 - np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))))) # map from (0,1) -> [1,19]

def jitter(size,const=0):
    jit = np.random.normal(0+const, 0.05, size)
    X = np.zeros((size))
    X = X + jit
    return X

def other(target):
    other_objs = [i for i in ['bed', 'bench', 'chair', 'table'] if i not in target]
    return other_objs

def red_vox(n_vox, prop=0.1):
    return int(np.ceil(n_vox * prop))

def get_inds(X, Y, pair, testRun=None):

    inds = {}

    # return relative indices
    if testRun:
        trainIX = Y.index[(Y['label'].isin(pair)) & (Y['run_num'] != int(testRun))]
    else:
        trainIX = Y.index[(Y['label'].isin(pair))]

    # pull training and test data
    trainX = X[trainIX]
    trainY = Y.iloc[trainIX].label

    # Main classifier on 5 runs, testing on 6th
    clf = LogisticRegression(penalty='l2',C=1, solver='lbfgs', max_iter=1000, 
                            multi_class='multinomial').fit(trainX, trainY)
    B = clf.coef_[0]  # pull betas

    # retrieve only the first object, then only the second object
    if testRun:
        obj1IX = Y.index[(Y['label'] == pair[0]) & (Y['run_num'] != int(testRun))]
        obj2IX = Y.index[(Y['label'] == pair[1]) & (Y['run_num'] != int(testRun))]
    else:
        obj1IX = Y.index[(Y['label'] == pair[0])]
        obj2IX = Y.index[(Y['label'] == pair[1])]

    # Get the average of the first object, then the second object
    obj1X = np.mean(X[obj1IX], 0)
    obj2X = np.mean(X[obj2IX], 0)

    # Build the importance map
    mult1X = obj1X * B
    mult2X = obj2X * B

    # Sort these so that they are from least to most important for a given category.
    sortmult1X = mult1X.argsort()[::-1]
    sortmult2X = mult2X.argsort()

    # add to a dictionary for later use
    inds[clf.classes_[0]] = sortmult1X
    inds[clf.classes_[1]] = sortmult2X

    return inds

if 'milgram' in os.getcwd():
    main_dir='/gpfs/milgram/project/turk-browne/projects/rtTest/'
else:
    main_dir='/Users/kailong/Desktop/rtTest'

working_dir=main_dir
os.chdir(working_dir)

objects = ['bed', 'bench', 'chair', 'table']


if dataSource == "neurosketch":
    funcdata = "/gpfs/milgram/project/turk-browne/jukebox/ntb/projects/sketchloop02/subjects/{sub}_neurosketch/data/nifti/realtime_preprocessed/{sub}_neurosketch_recognition_run_{run}.nii.gz"
    metadata = "/gpfs/milgram/project/turk-browne/jukebox/ntb/projects/sketchloop02/data/features/recog/metadata_{sub}_V1_{phase}.csv"
    anat = "/gpfs/milgram/project/turk-browne/jukebox/ntb/projects/sketchloop02/subjects/{sub}_neurosketch/data/nifti/{sub}_neurosketch_anat_mprage_brain.nii.gz"
elif dataSource == "realtime":
    funcdata = "/gpfs/milgram/project/turk-browne/projects/rtcloud_kp/subjects/{sub}/ses{ses}_recognition/run0{run}/nifti/{sub}_functional.nii.gz"
    metadata = "/gpfs/milgram/project/turk-browne/projects/rtcloud_kp/subjects/{sub}/ses{ses}_recognition/run0{run}/{sub}_0{run}_preprocessed_behavData.csv"
    anat = "$TO_BE_FILLED"
else:
    funcdata = "/gpfs/milgram/project/turk-browne/projects/rtTest/searchout/feat/{sub}_pre.nii.gz"
    metadata = "/gpfs/milgram/project/turk-browne/jukebox/ntb/projects/sketchloop02/data/features/recog/metadata_{sub}_V1_{phase}.csv"
    anat = "$TO_BE_FILLED"

# print('mask dimensions: {}'. format(mask.shape))
# print('number of voxels in mask: {}'.format(np.sum(mask)))
phasedict = dict(zip([1,2,3,4,5,6],["12", "12", "34", "34", "56", "56"]))
imcodeDict={"A": "bed", "B": "Chair", "C": "table", "D": "bench"}
chosenMask = np.load(f"/gpfs/milgram/project/turk-browne/projects/rtTest/schaefer2018/{subject}/chosenMask.npy")
print(f"np.sum(chosenMask)={np.sum(chosenMask)}")
# Compile preprocessed data and corresponding indices
metas = []
for run in range(1, 7):
    print(run, end='--')
    # retrieve from the dictionary which phase it is, assign the session
    phase = phasedict[run]

    # Build the path for the preprocessed functional data
    this4d = funcdata.format(run=run, phase=phase, sub=subject)

    # Read in the metadata, and reduce it to only the TR values from this run, add to a list
    thismeta = pd.read_csv(metadata.format(run=run, phase=phase, sub=subject))
    if dataSource == "neurosketch":
        _run = 1 if run % 2 == 0 else 2
    else:
        _run = run
    thismeta = thismeta[thismeta['run_num'] == int(_run)]

    if dataSource == "realtime":
        TR_num = list(thismeta.TR.astype(int))
        labels = list(thismeta.Item)
        labels = [imcodeDict[label] for label in labels]
    else:
        TR_num = list(thismeta.TR_num.astype(int))
        labels = list(thismeta.label)

    print("LENGTH OF TR: {}".format(len(TR_num)))
    # Load the functional data
    runIm = nib.load(this4d)
    affine_mat = runIm.affine
    runImDat = runIm.get_fdata()

    # Use the TR numbers to select the correct features
    features = [runImDat[:,:,:,n+3] for n in TR_num] # here shape is from (94, 94, 72, 240) to (80, 94, 94, 72)
    features = np.array(features)
    features = features[:, chosenMask==1]
    print("shape of features", features.shape, "shape of chosenMask", chosenMask.shape)
    features = normalize(features)
    # features = np.expand_dims(features, 0)

    # Append both so we can use it later
    # metas.append(labels)
    # metas['label']

    t=pd.DataFrame()
    t['label']=labels
    t["run_num"]=run
    behav_data=t if run==1 else pd.concat([behav_data,t])

    runs = features if run == 1 else np.concatenate((runs, features))

dimsize = runIm.header.get_zooms()
brain_data = runs
print(brain_data.shape)
print(behav_data.shape)
FEAT=brain_data
print(f"FEAT.shape={FEAT.shape}")
META=behav_data

def Class(brain_data,behav_data):
    accs = []
    for run in range(1,7):
        trainIX = behav_data['run_num']!=int(run)
        testIX = behav_data['run_num']==int(run)

        trainX =  brain_data[trainIX]
        trainY =  behav_data.iloc[np.asarray(trainIX)].label

        testX =  brain_data[testIX]
        testY =  behav_data.iloc[np.asarray(testIX)].label

        clf = LogisticRegression(penalty='l2',C=1, solver='lbfgs', max_iter=1000, 
                                multi_class='multinomial').fit(trainX, trainY)

        # Monitor progress by printing accuracy (only useful if you're running a test set)
        acc = clf.score(testX, testY)
        accs.append(acc)
    accs
    return np.mean(accs)
accs=Class(brain_data,behav_data)
print(f"new trained 4 way classifier accuracy={accs}")


# convert item colume to label colume
imcodeDict={
'A': 'bed',
'B': 'chair',
'C': 'table',
'D': 'bench'}

# Which run to use as test data (leave as None to not have test data)
accs_rotation_container={}
evidence_rotation_container={}
for testRun in range(1,7):
    # testRun = 6 # when testing: testRun = 2 ; META['run_num'].iloc[:5]=2

    # Decide on the proportion of crescent data to use for classification
    include = 1
    objects = ['bed', 'bench', 'chair', 'table']
    allpairs = itertools.combinations(objects,2)
    accs={}
    # Iterate over all the possible target pairs of objects
    for pair in allpairs:
        # Find the control (remaining) objects for this pair
        altpair = other(pair)

        # pull sorted indices for each of the critical objects, in order of importance (low to high)
        # inds = get_inds(FEAT, META, pair, testRun=testRun)

        # Find the number of voxels that will be left given your inclusion parameter above
        # nvox = red_vox(FEAT.shape[1], include)

        for obj in pair:
            # foil = [i for i in pair if i != obj][0]
            for altobj in altpair:

                # establish a naming convention where it is $TARGET_$CLASSIFICATION
                # Target is the NF pair (e.g. bed/bench)
                # Classificationis is btw one of the targets, and a control (e.g. bed/chair, or bed/table, NOT bed/bench)
                naming = '{}{}_{}{}'.format(pair[0], pair[1], obj, altobj)

                # Pull the relevant inds from your previously established dictionary 
                # obj_inds = inds[obj]

                # If you're using testdata, this function will split it up. Otherwise it leaves out run as a parameter
                # if testRun:
                #     trainIX = META.index[(META['label'].isin([obj, altobj])) & (META['run_num'] != int(testRun))]
                #     testIX = META.index[(META['label'].isin([obj, altobj])) & (META['run_num'] == int(testRun))]
                # else:
                #     trainIX = META.index[(META['label'].isin([obj, altobj]))]
                #     testIX = META.index[(META['label'].isin([obj, altobj]))]
                # # pull training and test data
                # trainX = FEAT[trainIX]
                # testX = FEAT[testIX]
                # trainY = META.iloc[trainIX].label
                # testY = META.iloc[testIX].label

                # print(f"obj={obj},altobj={altobj}")
                # print(f"unique(trainY)={np.unique(trainY)}")
                # print(f"unique(testY)={np.unique(testY)}")
                # assert len(np.unique(trainY))==2

                # for testRun in range(6):
                if testRun:
                    trainIX = ((META['label']==obj) + (META['label']==altobj)) * (META['run_num']!=int(testRun))
                    testIX = ((META['label']==obj) + (META['label']==altobj)) * (META['run_num']==int(testRun))
                else:
                    trainIX = ((META['label']==obj) + (META['label']==altobj))
                    testIX = ((META['label']==obj) + (META['label']==altobj))
                # pull training and test data
                trainX = FEAT[trainIX]
                testX = FEAT[testIX]
                trainY = META.iloc[np.asarray(trainIX)].label
                testY = META.iloc[np.asarray(testIX)].label

                # print(f"obj={obj},altobj={altobj}")
                # print(f"unique(trainY)={np.unique(trainY)}")
                # print(f"unique(testY)={np.unique(testY)}")
                assert len(np.unique(trainY))==2

                # # If you're selecting high-importance features, this bit handles that
                # if include < 1:
                #     trainX = trainX[:, obj_inds[-nvox:]]
                #     testX = testX[:, obj_inds[-nvox:]]

                # Train your classifier
                clf = LogisticRegression(penalty='l2',C=1, solver='lbfgs', max_iter=1000, 
                                        multi_class='multinomial').fit(trainX, trainY)


                model_folder = f"{working_dir}{roiloc}/{subject}/clf/"
                mkdir(model_folder)
                # Save it for later use
                joblib.dump(clf, model_folder +'/{}.joblib'.format(naming))

                # Monitor progress by printing accuracy (only useful if you're running a test set)
                acc = clf.score(testX, testY)
                # print(naming, acc)
                accs[naming]=acc
    accs_rotation_container[testRun] = accs
    evidence_rotation_container[testRun] = morphingTarget(subject,testRun=testRun)
for testRun in range(1,7):
    t=np.mean(list(accs_rotation_container[testRun].values()))
    print(f"testRun {testRun} mean={t}")
    print(evidence_rotation_container[testRun][2])
# _=plt.figure()
# _=plt.hist(list(accs.values()))

conda env=/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud
np.sum(chosenMask)=3140
1--LENGTH OF TR: 80
shape of features (80, 3140) shape of chosenMask (94, 94, 72)
2--LENGTH OF TR: 80
shape of features (80, 3140) shape of chosenMask (94, 94, 72)
3--LENGTH OF TR: 80
shape of features (80, 3140) shape of chosenMask (94, 94, 72)
4--LENGTH OF TR: 80
shape of features (80, 3140) shape of chosenMask (94, 94, 72)
5--LENGTH OF TR: 80
shape of features (80, 3140) shape of chosenMask (94, 94, 72)
6--LENGTH OF TR: 80
shape of features (80, 3140) shape of chosenMask (94, 94, 72)
(480, 3140)
(480, 2)
FEAT.shape=(480, 3140)
new trained 4 way classifier accuracy=0.3583333333333334
1--LENGTH OF TR: 80
shape of features (80, 3140) shape of mask (94, 94, 72)
(80, 3140)
(80, 2)
FEAT.shape=(80, 3140)
floor
D evidence for AD_clf when A is presented=0.047310039634387735
C evidence for AC_clf when A is presented=-0.11202338889783711
D evidence for CD_clf when A is presented=0.18090837724675982
C 

3--LENGTH OF TR: 80
shape of features (80, 3140) shape of mask (94, 94, 72)
(80, 3140)
(80, 2)
FEAT.shape=(80, 3140)
floor
D evidence for AD_clf when A is presented=-0.3621483375428405
C evidence for AC_clf when A is presented=-0.2749235228322624
D evidence for CD_clf when A is presented=-0.22105531329652503
C evidence for CD_clf when A is presented=0.22105531329652503
ceil
A evidence in AC_clf when A is presented=0.2749235228322624
A evidence in AD_clf when A is presented=0.3621483375428405
evidence_ceil=0.3185359301875514
floor=0.22105531329652503, ceil=0.3185359301875514
mu=0.2697956217420382, sig=0.041396558897157446
classifierEvidence(BC_clf,FEAT,Y)=[[-0.48806756]
 [ 0.17926902]
 [ 0.17863708]
 [-2.54582776]
 [-2.12132613]
 [-2.13610597]
 [-1.24992135]
 [ 2.15392019]
 [-0.51096872]
 [ 1.20431702]
 [-0.05940045]
 [-0.67407038]
 [-1.56750567]
 [ 0.51561134]
 [-0.23938004]
 [ 0.43617273]
 [ 1.13977441]
 [-2.01036437]
 [-0.81688321]
 [-0.74212555]
 [-0.25125048]
 [ 0.14309484]
 [ 0.53

5--LENGTH OF TR: 80
shape of features (80, 3140) shape of mask (94, 94, 72)
(80, 3140)
(80, 2)
FEAT.shape=(80, 3140)
floor
D evidence for AD_clf when A is presented=-0.35312844580640734
C evidence for AC_clf when A is presented=-0.08338697586240863
D evidence for CD_clf when A is presented=-0.28104307793522487
C evidence for CD_clf when A is presented=0.28104307793522487
ceil
A evidence in AC_clf when A is presented=0.08338697586240863
A evidence in AD_clf when A is presented=0.35312844580640734
evidence_ceil=0.21825771083440793
floor=0.28104307793522487, ceil=0.21825771083440793
mu=0.2496503943848164, sig=-0.026662717471045077
classifierEvidence(BC_clf,FEAT,Y)=[[-2.32727276]
 [ 1.81723576]
 [-0.19526798]
 [ 1.25039423]
 [-0.82713093]
 [ 0.28549732]
 [ 0.0574965 ]
 [ 1.1749786 ]
 [ 0.38147128]
 [-0.0365044 ]
 [-1.24695928]
 [ 0.42708951]
 [ 0.02532819]
 [ 0.40474552]
 [ 0.64036901]
 [ 0.09246172]
 [-1.25406899]
 [ 0.53956835]
 [ 1.50837271]
 [-0.03789663]
 [ 0.99392019]
 [-1.54025337]


In [39]:
for testRun in range(1,7):
    t=np.mean(list(accs_rotation_container[testRun].values()))
    print(f"testRun {testRun} mean testing accracy={t}",end="")
    print(evidence_rotation_container[testRun][2])
    print()
    print()

testRun 1 mean testing accracy=0.5958333333333333
D evidence for AD_clf when A is presented=0.047310039634387735
C evidence for AC_clf when A is presented=-0.11202338889783711
D evidence for CD_clf when A is presented=0.18090837724675982
C evidence for CD_clf when A is presented=-0.18090837724675982
ceil
A evidence in AC_clf when A is presented=0.11202338889783711
A evidence in AD_clf when A is presented=-0.047310039634387735
evidence_ceil=0.03235667463172471
floor=-0.18090837724675982, ceil=0.03235667463172471
mu=-0.07427585130751756, sig=0.09056609982949063
morphParam for bed = 17.35
morphParam for chair = 16.85
morphParam for table = 17.75
morphParam for bench = 17.5


testRun 2 mean testing accracy=0.6208333333333332
D evidence for AD_clf when A is presented=0.12889209589277678
C evidence for AC_clf when A is presented=0.137339553488769
D evidence for CD_clf when A is presented=-0.147249505997735
C evidence for CD_clf when A is presented=0.147249505997735
ceil
A evidence in AC_clf 

In [51]:
def getString(string,searchFor,end):
    return string.split(searchFor)[1].split(end)[0]
# getString(string,searchFor)

floors=[]
ceils=[]
for testRun in range(1,7):
    t=np.mean(list(accs_rotation_container[testRun].values()))
    print(f"testRun {testRun} mean testing accracy={t}")
    # print(evidence_rotation_container[testRun][2])
    t=np.float(getString(evidence_rotation_container[testRun][2],"floor=",","))
    print("floor=",t)
    floors.append(t)
    
    t=np.float(getString(evidence_rotation_container[testRun][2],"ceil=","\n"))
    print("ceiling=",t)
    ceils.append(t)
    print()
    print()
print(np.mean(floors))
print(np.mean(ceils))

testRun 1 mean testing accracy=0.5958333333333333
floor= -0.18090837724675982
ceiling= 0.03235667463172471


testRun 2 mean testing accracy=0.6208333333333332
floor= 0.147249505997735
ceiling= -0.13311582469077288


testRun 3 mean testing accracy=0.5875
floor= 0.22105531329652503
ceiling= 0.3185359301875514


testRun 4 mean testing accracy=0.5375
floor= -0.009144018258912623
ceiling= 0.0702778408002945


testRun 5 mean testing accracy=0.5375
floor= 0.28104307793522487
ceiling= 0.21825771083440793


testRun 6 mean testing accracy=0.5708333333333333
floor= 0.08732330754110029
ceiling= -0.024526959849264375


0.09110313487748546
0.08029756198565688


In [60]:

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
training_data, binary_labels=X[:100],y[:100]

# clf = LogisticRegression(random_state=0).fit(X, y)
# clf.predict(X[:2, :])

# clf.predict_proba(X[:2, :])


# clf.score(X, y)


lr = LogisticRegression()
lr.fit(training_data, binary_labels)

# Generate probabities automatically
# predicted_probs = lr.predict_proba(binary_labels)
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))
# sigmoid( dot([val1, val2, offset], lr.coef_.T) )
np.dot(training_data, lr.coef_.T) + lr.intercept_
# sigmoid( np.dot(training_data, lr.coef_.T) + lr.intercept_ )

array([[-4.11581963],
       [-3.75040852],
       [-4.25072468],
       [-3.74236522],
       [-4.25055292],
       [-3.46148321],
       [-4.14907246],
       [-3.83830973],
       [-3.87989412],
       [-3.70648853],
       [-3.93425424],
       [-3.69553313],
       [-3.89067777],
       [-4.80340887],
       [-4.72274745],
       [-4.24455696],
       [-4.38488147],
       [-4.01958686],
       [-3.33490971],
       [-4.06082773],
       [-3.20046467],
       [-3.87389816],
       [-5.35009711],
       [-2.95317901],
       [-3.00298443],
       [-3.24467291],
       [-3.41499464],
       [-3.84093358],
       [-3.98108633],
       [-3.55817598],
       [-3.42344269],
       [-3.46969828],
       [-4.48134722],
       [-4.57455139],
       [-3.61025577],
       [-4.3494648 ],
       [-4.17052326],
       [-4.39082216],
       [-4.2014405 ],
       [-3.79427325],
       [-4.29447291],
       [-3.42629357],
       [-4.38283412],
       [-3.31322592],
       [-3.04119671],
       [-3

In [61]:
lr.predict_proba(training_data)
# def classifierEvidence(clf,X,Y):
#     ID=np.where((clf.classes_==Y)*1==1)[0][0]
#     p = clf.predict_proba(X)[:,ID]
#     BX=np.log(p/(1-p))
#     return BX
def logit(p):
    np.log(p/(1-p))

array([[0.98394926, 0.01605074],
       [0.9770318 , 0.0229682 ],
       [0.98594642, 0.01405358],
       [0.97685061, 0.02314939],
       [0.98594404, 0.01405596],
       [0.96957176, 0.03042824],
       [0.98446607, 0.01553393],
       [0.97892381, 0.02107619],
       [0.9797649 , 0.0202351 ],
       [0.97602528, 0.02397472],
       [0.98081498, 0.01918502],
       [0.97576758, 0.02423242],
       [0.97997759, 0.02002241],
       [0.99186498, 0.00813502],
       [0.99118763, 0.00881237],
       [0.9858607 , 0.0141393 ],
       [0.98768908, 0.01231092],
       [0.9823565 , 0.0176435 ],
       [0.96560719, 0.03439281],
       [0.98305726, 0.01694274],
       [0.96085176, 0.03914824],
       [0.97964569, 0.02035431],
       [0.99527474, 0.00472526],
       [0.95041352, 0.04958648],
       [0.95270877, 0.04729123],
       [0.96248122, 0.03751878],
       [0.96816988, 0.03183012],
       [0.97897787, 0.02102213],
       [0.98167666, 0.01832334],
       [0.97229849, 0.02770151],
       [0.